In [1]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.feature import hog
from PIL import Image
import numpy as np
import random
import pickle
import glob
import time
import cv2

Wall time: 1.52 s


In [3]:
%%time
#time function always at the beginning of the code block
""" step 1 : reading the images """
print(' Reading vehicles images...')
vehicle_images = []

GTI_far = glob.glob('./data/data1/vehicles/GTI_Far/*.png')
GTI_left = glob.glob('./data/data1/vehicles/GTI_Left/*.png')
GTI_middleClose = glob.glob('./data/data1/vehicles/MiddleClose/*.png')
GTI_right = glob.glob('./data/data1/vehicles/GTI_Right/*.png')
GTI_extracted = glob.glob('./data/data1/vehicles/KITTI_exctracted//*.png')
cars_img = glob.glob('./data/car_data/train/A*/*.jpg')
cars_img2 = glob.glob('./data/car_data/train/B*/*.jpg')
cars_img3 = glob.glob('./data/car_data/train/C*/*.jpg')

vehicle_images = np.concatenate([GTI_far, GTI_left, GTI_middleClose, GTI_right, GTI_extracted, cars_img, cars_img2, cars_img3])

vehicle_images_original = []
for imagePath in vehicle_images:
    readImg = cv2.imread(imagePath)
    Img = cv2.cvtColor(readImg, cv2.COLOR_BGR2GRAY)
    Img = cv2.resize(Img,(32, 32))
    vehicle_images_original.append(Img)
    
print(' Done')

vehicles_img = np.asarray(vehicle_images_original)
print(vehicles_img.shape)

 Reading vehicles images...
 Done
(5738, 32, 32)
Wall time: 31.6 s


In [4]:
%%time
print(' Reading non-vehicles images...')
non_vehicles_images = []
Extras = glob.glob('./data/non-vehicles/*.jpg')
#extras2 = glob.glob('./data/data1/non-vehicles/Extras//*.png')
GTI = glob.glob('./data/data1/non-vehicles/GTI/*.png')

non_vehicle_images = np.concatenate([Extras, GTI])

non_vehicle_images_original = []
for imagePath in non_vehicle_images:
    readImg = cv2.imread(imagePath)
    Img = cv2.cvtColor(readImg, cv2.COLOR_BGR2GRAY)
    Img = cv2.resize(Img,(32, 32))
    non_vehicle_images_original.append(Img)
    
print(' Done')

non_vehicles_img = np.asarray(non_vehicle_images_original)
print(non_vehicles_img.shape)

 Reading non-vehicles images...
 Done
(4115, 32, 32)
Wall time: 33.9 s


In [5]:
%%time
print("N° of vehicle images loaded : " + str(len(vehicle_images_original)))
print("N° of non-vehicle images loaded : " + str(len(non_vehicle_images_original)))

N° of vehicle images loaded : 5738
N° of non-vehicle images loaded : 4115
Wall time: 998 µs


In [6]:
%%time
""" HOG functions """
# Return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        hog_features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  visualize=vis, feature_vector=feature_vec)
        return hog_features, hog_image
    # Otherwise call with one output
    else:      
        hog_features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       visualize=vis, feature_vector=feature_vec)
        return hog_features

# Return extracted features 
def ExtractFeatures(images, orient, pix_per_cell, cell_per_block):
    featureList = []
    for Img in images:
        local_features = get_hog_features(Img, orient, pix_per_cell, cell_per_block, False, True)
        featureList.append(local_features)
    return featureList


Wall time: 0 ns


In [7]:
%%time
# can be used to visualize confusion matrix if needed
# Extract features for vehicle images 
vehicleFeatures= ExtractFeatures(vehicle_images_original, 10, 8, 4)

#vehicleFeat = np.asarray(vehicleFeatures)

Wall time: 2.11 s


In [8]:
%%time
# Extract features for non-vehicle images
nonVehicleFeatures = ExtractFeatures(non_vehicle_images_original, 10, 8, 4)

#nonVehicleFeat = np.asarray(nonVehicleFeatures)

Wall time: 1.32 s


In [9]:
%%time
featuresList= np.vstack([vehicleFeatures, nonVehicleFeatures])
print("Shape of features list is ", featuresList.shape)
labelList= np.concatenate([np.ones(len(vehicleFeatures)), np.zeros(len(nonVehicleFeatures))])
print("Shape of the label list is ", labelList.shape)

Shape of features list is  (9853, 160)
Shape of the label list is  (9853,)
Wall time: 216 ms


In [10]:
%%time
""" Data Preprocessing """
# train test split of data
X_train,  X_test,Y_train, Y_test = train_test_split(featuresList, labelList, test_size=0.2, shuffle=True)
print(X_train.shape)
print(X_test.shape)

(7882, 160)
(1971, 160)
Wall time: 192 ms


In [11]:
%%time
#normalization and scaling

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)

print(X_train_scaled.shape)
print(X_test_scaled.shape)

(7882, 160)
(1971, 160)
Wall time: 293 ms


In [12]:
%%time
""" Define and Train a classifier """
from sklearn.svm import LinearSVC
classifier1= LinearSVC()
classifier1.fit(X_train,Y_train)
print("test Accuracy is  ", classifier1.score(X_test,Y_test) )
print("training Accuracy is  ", classifier1.score(X_train,Y_train) )

test Accuracy is   0.9528158295281582
training Accuracy is   0.9663790916011165
Wall time: 11 s


In [13]:
%%time
filename = 'classifier14.xml'
pickle.dump(classifier1, open(filename, 'wb'))

Wall time: 1 ms
